<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [14]:
import sxdm
import h5py

In [20]:
pwd

'/home/esrf/zatterin/repos/id01-sxdm-utils/tests'

In [15]:
path_dset = './data/sample/sample_0001/sample_0001.h5'

In [16]:
test = sxdm.widgets.bliss.Inspect4DSXDM(path_dset, '1.1')
test.show()

In [9]:
with h5py.File(path_dset) as h5f:
    arr = h5f['1.1/measurement/mpx1x4'][()]
    arr = arr.reshape(60,60,516,516)

In [13]:
test = sxdm.widgets.Inspect4DArray(arr)
test.show()